# Exercise Sheet 8

## Exercise 1: Shapley Values

In [1]:
import numpy as np
import math
import itertools

In [2]:
np.random.seed(123456)

## a)

In [3]:
def payoff(coalition):
    # Define boolean variables that indicate whether Timnit, Margret, Samy, Jeff and Larry are in the set or not.
    t = 't' in coalition
    s = 's' in coalition
    m = 'm' in coalition
    j = 'j' in coalition
    l = 'l' in coalition
    return 10*t + 10*m + 10*s + 2*j + 20 * (t and m) + 20 * (t and m and s) - 30 * ((t or m or s) and j)

In [4]:
# Testing
population = ['j', 'l', 'm', 's', 't']
print(payoff(('t','m')))
print(payoff(['t','j','s']))
print(payoff([]))
print(payoff(population))

40
-8
0
42


## b)

In [5]:
def all_unique_subsets(population):
    if not population:
        return [[]]
    else:
        population = list(set(population)) # remove double elements
        subsets_wo_member = all_unique_subsets(population[1:])
        subsets_w_member = [s + [population[0]] for s in subsets_wo_member]
        return subsets_w_member + subsets_wo_member

In [6]:
# Official solution: https://docs.python.org/2/library/itertools.html#recipes
# def all_unique_subsets(population):
#    population = list(set(population))
#    power_set = list(itertools.chain.from_iterable(itertools.combinations(population, r) for r in range(len(population)+1)))
#    return [list(subset) for subset in power_set]

In [7]:
# Testing
print(all_unique_subsets([]))
print(all_unique_subsets(['t']))
print(all_unique_subsets(population))

[[]]
[['t'], []]
[['t', 'l', 'j', 'm', 's'], ['l', 'j', 'm', 's'], ['t', 'j', 'm', 's'], ['j', 'm', 's'], ['t', 'l', 'm', 's'], ['l', 'm', 's'], ['t', 'm', 's'], ['m', 's'], ['t', 'l', 'j', 's'], ['l', 'j', 's'], ['t', 'j', 's'], ['j', 's'], ['t', 'l', 's'], ['l', 's'], ['t', 's'], ['s'], ['t', 'l', 'j', 'm'], ['l', 'j', 'm'], ['t', 'j', 'm'], ['j', 'm'], ['t', 'l', 'm'], ['l', 'm'], ['t', 'm'], ['m'], ['t', 'l', 'j'], ['l', 'j'], ['t', 'j'], ['j'], ['t', 'l'], ['l'], ['t'], []]


In [8]:
def shapley_set(member, population, v_function=payoff, *args):
    population = list(set(population)) # remove double elements
    remainder = population[:] # copy
    remainder.remove(member)
    all_coalitions_without_member = all_unique_subsets(remainder)
    result = 0
    F = len(population)
    for coalition in all_coalitions_without_member:
        S = len(coalition)
        diff = v_function(coalition + [member], *args) - v_function(coalition, *args)
        factor = math.factorial(S) * math.factorial(F-S-1) / math.factorial(F)
        result += factor * diff
    return result

In [9]:
# Testing and Sanity Check
def all_shapley_values(population, shapley_fct, v_function=payoff, *args):
    return {member:shapley_fct(member, population, v_function, *args) for member in population}

def shapley_test(population, shapley_fct, v_function=payoff, *args):
    shapley_values=all_shapley_values(population, shapley_fct, v_function, *args)
    print("\n".join(f"Player {player}: {shapley_values[player]}" for player in shapley_values))
    sum_ = sum(shapley_values.values())
    print(sum_)
    print(v_function(population, *args))
    print(math.isclose(sum_, v_function(population, *args))) # efficiency axiom

In [10]:
shapley_test(population, shapley_set)

Player j: -20.5
Player l: 0.0
Player m: 24.166666666666668
Player s: 14.16666666666667
Player t: 24.166666666666664
42.0
42
True


### Definition of a few useful functions

In [11]:
def all_coalitions_without_members(members, population):
    population_without_members = population.copy()
    population_without_members.remove(members)
    return all_unique_subsets(population_without_members)

def payoff_diff(member, coalition, v_function=payoff, *args):
    return v_function(coalition + [member], *args) - v_function(coalition, *args)

def payoff_diff_list(member, coalitions, v_function=payoff, *args):
    return np.asarray([payoff_diff(member, coalition, v_function, *args) for coalition in coalitions])

In [12]:
def shapley_set_2(member, population, v_function=payoff, *args):
    population = list(set(population)) # remove double elements
    all_coalitions_without_member = all_coalitions_without_members(member, population)

    payoff_diffs = payoff_diff_list(member, all_coalitions_without_member, v_function, *args)

    F = len(population)
    weights = np.asarray([math.factorial(len(coalition)) * math.factorial(F - len(coalition) - 1) / math.factorial(F)
                          for coalition in all_coalitions_without_member])
    return np.sum(weights*payoff_diffs)

In [13]:
# Testing and Sanity Check
shapley_test(population, shapley_set_2)

Player j: -20.5
Player l: 0.0
Player m: 24.166666666666664
Player s: 14.166666666666666
Player t: 24.166666666666668
42.0
42
True


## c)

In [14]:
def shapley_perm(member, population, v_function=payoff, *args):
    # Note that itertools.permutations returns tuples, which need to be converted to lists first
    # permutation.index(member) returns the index of member in the permutation, therfore
    # permutation[:permutation.index(member)] is exactly the permutation before this member
    coalitions = [permutation[:permutation.index(member)] for permutation in itertools.permutations(population)]
    return np.mean([payoff_diff(member, list(coalition), v_function, *args) for coalition in coalitions])

In [15]:
# Testing and Sanity Check
shapley_test(population, shapley_perm)

Player j: -20.5
Player l: 0.0
Player m: 24.166666666666668
Player s: 14.166666666666666
Player t: 24.166666666666668
42.0
42
True


## d)

In [16]:
def shapley_perm_approx(member, population, v_function=payoff, *args, its=100):
    vals = []
    rng = np.random.default_rng()
    for i in range(its):
        permutation = list(rng.permutation(population))
        coalition = permutation[:permutation.index(member)]
        vals.append(payoff_diff(member, coalition, v_function, *args))
    return np.mean(vals)

In [17]:
# Testing
for its in [1000, 10000, 10**5, 10**6]:
    print(shapley_perm_approx('t', population, its=its))
shapley_test(population, shapley_perm_approx)
shapley_test(population, lambda *args: shapley_perm_approx(*args, its=100000))

24.57
24.036
24.0476
24.17616
Player j: -21.7
Player l: 0.0
Player m: 26.4
Player s: 14.8
Player t: 23.8
43.3
42
False
Player j: -20.4538
Player l: 0.0
Player m: 24.1379
Player s: 14.0591
Player t: 24.1913
41.9345
42
False


Clear convergence is apparent, but it is not very fast, although with 100000 iterations we already get quite a good approximation.

The exact numbers here depend of course on the specific parameters, the seed etc ...

## e) 

### i) Symmetry Check

In [18]:
def symmetry_check(j, k, population, shapley_func, v_function, *args):
    remainder = set(population) - set([j, k])
    all_S = all_unique_subsets(remainder)
    surpluss_j = []
    surpluss_k = []
    for S in all_S:
        surplus_j = v_function(S + [j], *args) - v_function(S, *args)
        surplus_k = v_function(S + [k], *args) - v_function(S, *args)
        surpluss_j.append(surplus_j)
        surpluss_k.append(surplus_k)
    surpluss_j, surpluss_k = np.array(surpluss_j), np.array(surpluss_k)
    equal_surplus = np.all(surpluss_j == surpluss_k)
    if equal_surplus:
        print('equal surplus')
        val_j = shapley_func(j, population, v_function, *args)
        val_k = shapley_func(k, population, v_function, *args)
        return val_j == val_k
    else:
        return True


In [19]:
symmetry_check('m', 't', population, shapley_set_2, payoff)

equal surplus


np.False_

### ii) Dummy property check

In [20]:
def dummy_check(j, population, shapley_func, v_function, *args):
    remainder = set(population) - set([j])
    all_S = all_unique_subsets(remainder)
    surpluss_j = []
    for S in all_S:
        surplus_j = v_function(S + [j], *args) - v_function(S, *args)
        surpluss_j.append(surplus_j)
    has_contribution = np.sum(np.abs(surpluss_j)) > 0
    if has_contribution:
        print('has contribution')
        val_j = shapley_func(j, population, v_function, *args)
        return val_j > 0
    else:
        return True

In [21]:
dummy_check('l', population, shapley_set_2, payoff)

True

### iii) Additivity check

In [22]:
def additivity_check(population, shapley_func, v_function1, v_function2, *args):
    combined = lambda x : v_function1(x, *args) + v_function2(x, *args)
    vals1 = np.array([shapley_func(j, population, v_function1, *args) for j in population])
    vals2 = np.array([shapley_func(j, population, v_function2, *args) for j in population])
    vals_comb = np.array([shapley_func(j, population, combined) for j in population])
    vals_additive = vals1 + vals2
    return np.all(vals_comb == vals_additive)


In [23]:
payoff2 = payoff

additivity_check(population, shapley_set_2, payoff, payoff2)

np.True_

### iv): Efficiency check

In [24]:
def efficiency_check(population, shapley_func, v_function, *args):
    payoff_total = v_function(population, *args)
    shapley_vals = [shapley_func(j, population, v_function, *args) for j in population]
    total_shapley_vals = np.sum(shapley_vals)
    pt, st = round(payoff_total, 5), round(total_shapley_vals, 5)
    return pt == st

In [25]:
efficiency_check(population, shapley_set_2, payoff)

np.True_